# Likelihood Serialization and Patches

## Using HEPData

### Preserved on HEPData

As of this tutorial, ATLAS has published 4 full likelihoods to HEPData

<p align="center">
<a href="https://www.hepdata.net/record/ins1755298?version=3"><img src="https://raw.githubusercontent.com/matthewfeickert/talk-SciPy-2020/e0c509cd0dfef98f5876071edd4c60aff9199a1b/figures/HEPData_likelihoods.png"></a>
</p>

Let's explore the 1Lbb workspace a little bit shall we?

### Getting the Data

We'll use python `requests` and `tarfile` to download the tarball and extract the files we need.

In [1]:
import requests
import hashlib
import tarfile
import json

electroweakino_HEPData_URL = "https://www.hepdata.net/record/resource/1408476?view=true"
targz_filename = "data/1Lbb_workspaces.tar.gz"
response = requests.get(electroweakino_HEPData_URL, stream=True)
assert response.status_code == 200
with open(targz_filename, "wb") as file:
    file.write(response.content)
hashlib.sha256(open(targz_filename, "rb").read()).hexdigest()
# Open as a tarfile
tar_archive = tarfile.open(targz_filename, "r:gz")

### Instantiate our objects

To make this easier on us, we'll first define a nice helper function to extract out a file from the tarball.

In [2]:
def get_json_from_tarfile(tarfile, json_name):
    json_file = tarfile.extractfile(tarfile.getmember(json_name)).read().decode("utf8")
    return json.loads(json_file)

Let's go ahead and `import pyhf` and get started.

In [3]:
import pyhf

spec = get_json_from_tarfile(tar_archive, "BkgOnly.json")
patchset = pyhf.PatchSet(get_json_from_tarfile(tar_archive, "patchset.json"))

So what did the analyzers give us for signal patches?

## Patching in Signals

Let's look at this [`pyhf.PatchSet`](https://scikit-hep.org/pyhf/_generated/pyhf.patchset.PatchSet.html#pyhf.patchset.PatchSet) object which provides a user-friendly way to interact with many signal patches at once.

### PatchSet

In [4]:
patchset

<pyhf.patchset.PatchSet object with 125 patches at 0x7fb112485350>

Oh wow, we've got 125 patches. What information does it have?

In [ ]:
print(f'description: {patchset.description}')
print(f'    digests: {patchset.digests}')
print(f'     labels: {patchset.labels}')
print(f' references: {patchset.references}')
print(f'    version: {patchset.version}')

So we've got a useful description of the signal patches... there's a digest. Does that match the background-only workspace we have?

In [5]:
pyhf.utils.digest(spec)

'2563672e1a165384faf49f1431e921d88c9c07ec10f150d5045576564f98f820'

It does! In fact, this sort of verification check will be done automatically when applying patches using `pyhf.PatchSet` as we will see shortly. To manually verify, simply run `pyhf.PatchSet.verify` on the workspace. No error means everything is fine. It will loudly complain otherwise.

In [6]:
patchset.verify(spec)

No error, whew. Let's move on.

The labels `m1` and `m2` tells us that we have the signal patches parametrized in 2-dimensional space, likely as $m_1 = \tilde{\chi}_1^\pm$ and $m_2 = \tilde{\chi}_1^0$... but I guess we'll see?

The references list the references for this dataset, which is pointing at the hepdata record for now.

Next, the version is the version of the schema set we're using with `pyhf` (`1.0.0`).

And last, but certainly not least... its patches:

In [7]:
patchset.patches

[<pyhf.patchset.Patch object 'C1N2_Wh_hbb_1000_0(1000, 0)' at 0x7fb17c589b90>,
 <pyhf.patchset.Patch object 'C1N2_Wh_hbb_1000_100(1000, 100)' at 0x7fb11c309090>,
 <pyhf.patchset.Patch object 'C1N2_Wh_hbb_1000_150(1000, 150)' at 0x7fb1124857d0>,
 <pyhf.patchset.Patch object 'C1N2_Wh_hbb_1000_200(1000, 200)' at 0x7fb1124858d0>,
 <pyhf.patchset.Patch object 'C1N2_Wh_hbb_1000_250(1000, 250)' at 0x7fb112485c10>,
 <pyhf.patchset.Patch object 'C1N2_Wh_hbb_1000_300(1000, 300)' at 0x7fb112485990>,
 <pyhf.patchset.Patch object 'C1N2_Wh_hbb_1000_350(1000, 350)' at 0x7fb112485710>,
 <pyhf.patchset.Patch object 'C1N2_Wh_hbb_1000_400(1000, 400)' at 0x7fb112485d50>,
 <pyhf.patchset.Patch object 'C1N2_Wh_hbb_1000_50(1000, 50)' at 0x7fb112485810>,
 <pyhf.patchset.Patch object 'C1N2_Wh_hbb_150_0(150, 0)' at 0x7fb112485c90>,
 <pyhf.patchset.Patch object 'C1N2_Wh_hbb_165_35(165, 35)' at 0x7fb112485cd0>,
 <pyhf.patchset.Patch object 'C1N2_Wh_hbb_175_0(175, 0)' at 0x7fb11c2fbc10>,
 <pyhf.patchset.Patch obje

So we can see all the patches listed both by name such as `C1N2_Wh_hbb_900_250` as well as a pair of points `(900, 250)`. Why is this useful? The `PatchSet` object acts like a special dictionary look-up where it will grab the patch you need based on the unique key you provide it.

For example, we can look up by name

In [8]:
patchset['C1N2_Wh_hbb_900_250']

<pyhf.patchset.Patch object 'C1N2_Wh_hbb_900_250(900, 250)' at 0x7fb11249cd10>

or by the pair of points

In [9]:
patchset[(900, 250)]

<pyhf.patchset.Patch object 'C1N2_Wh_hbb_900_250(900, 250)' at 0x7fb11249cd10>

### Patches

A `pyhf.PatchSet` is a collection of `pyhf.Patch` objects. What is a patch indeed? It contains enough information about how to apply the signal patch to the corresponding background-only workspace (matched by digest).

In [10]:
patch = patchset['C1N2_Wh_hbb_900_250']
print(f'  name: {patch.name}')
print(f'values: {patch.values}')

  name: C1N2_Wh_hbb_900_250
values: (900, 250)


Most importantly, it contains the patch information itself. Specifically, this inherits from the `jsonpatch.JsonPatch` object, which is a 3rd party module providing native support for json patching in python. That means we can simply apply the patch to our workspace directly!

In [11]:
print(f' samples pre-patch: {pyhf.Workspace(spec).samples}')
print(f'samples post-patch: {pyhf.Workspace(patch.apply(spec)).samples}')

 samples pre-patch: ['diboson', 'multiboson', 'singletop', 'ttbar', 'tth', 'ttv', 'vh', 'wjets', 'zjets']
samples post-patch: ['C1N2_Wh_hbb_900_250', 'diboson', 'multiboson', 'singletop', 'ttbar', 'tth', 'ttv', 'vh', 'wjets', 'zjets']


Or, more quickly, from the `PatchSet` object:

In [12]:
print(f' samples pre-patch: {pyhf.Workspace(spec).samples}')
print(f'samples post-patch: {pyhf.Workspace(patchset.apply(spec, (900, 250))).samples}')

 samples pre-patch: ['diboson', 'multiboson', 'singletop', 'ttbar', 'tth', 'ttv', 'vh', 'wjets', 'zjets']
samples post-patch: ['C1N2_Wh_hbb_900_250', 'diboson', 'multiboson', 'singletop', 'ttbar', 'tth', 'ttv', 'vh', 'wjets', 'zjets']


### Patching via Model Creation

One last way to apply the patching is to, instead of patching workspaces, we patch the models as we build them from the background-only workspace. This maybe makes it easier to treat the background-only workspace as immutable, and patch in signal models when grabbing the model. Check it out.

In [13]:
workspace = pyhf.Workspace(spec)

First, load up our background-only spec into the workspace. Then let's create a model.

In [15]:
model = workspace.model(patches=[patchset['C1N2_Wh_hbb_900_250']])
print(f'samples (workspace): {workspace.samples}')
print(f'samples (  model  ): {model.config.samples}')

samples (workspace): ['diboson', 'multiboson', 'singletop', 'ttbar', 'tth', 'ttv', 'vh', 'wjets', 'zjets']
samples (  model  ): ['C1N2_Wh_hbb_900_250', 'diboson', 'multiboson', 'singletop', 'ttbar', 'tth', 'ttv', 'vh', 'wjets', 'zjets']


## Doing Physics

So we want to try and reproduce part of the contour. At least convince ourselves we're doing *physics* and not *fauxsics*. ... Anyway... Let's remind ourselves of the 1Lbb contour as we don't have the photographic memory of the ATLAS SUSY conveners

<img alt="1Lbb exclusion contour" src="https://atlas.web.cern.ch/Atlas/GROUPS/PHYSICS/PAPERS/SUSY-2019-08/fig_06.png" width="600" />

So let's work around the 700-900 GeV $\tilde{\chi}_1^\pm, \tilde{\chi}_2^0$ region. We'll look at two points here:

* `C1N2_Wh_hbb_650_0(650, 0)` which is below the contour and excluded
* `C1N2_Wh_hbb_1000_0(1000, 0)` which is above the contour and not excluded

Let's perform a "standard" hypothesis test (with $\mu = 1$ null BSM hypothesis) on both of these and use the $\text{CL}_\text{s}$ values to convince ourselves that we just did reproducible physics!?!

### Interpolators?

Ok, one last thing I should mention for the HistFitter users. We use nonlinear interpolators by default for the `normsys` and `histosys` modifiers. These correspond to `code4` and `code4p` in the [HistFactory paper](https://cds.cern.ch/record/1456844). We'll additionally configure the modifier settings to switch to these interpolators by default.

In [16]:
modifier_settings={
    "normsys": {"interpcode": "code4"},
    "histosys": {"interpcode": "code4p"},
}

### Doing Physics, for real now

In [17]:
model_below = workspace.model(
    patches=[patchset['C1N2_Wh_hbb_650_0']],
    modifier_settings=modifier_settings
)

model_above = workspace.model(
    patches=[patchset['C1N2_Wh_hbb_1000_0']],
    modifier_settings=modifier_settings
)

We've made our models. Let's test hypotheses!

*Note: this will not be as instantaneous as our simple models.. but it should still be pretty fast!*

In [18]:
result_below = pyhf.infer.hypotest(
    1.0,
    workspace.data(model_below),
    model_below,
    qtilde=True,
    return_expected_set=True
)
print(f"Observed: {result_below[0]}, Expected band: {[exp.tolist() for exp in result_below[1]]}")

/home/feickert/.venvs/louie-test/lib/python3.7/site-packages/pyhf/tensor/numpy_backend.py:271: RuntimeWarning: invalid value encountered in log
  return n * np.log(lam) - lam - gammaln(n + 1.0)


Observed: 0.01713749945809046, Expected band: [4.8418614774961446e-06, 9.133176119925303e-05, 0.0014659909896557642, 0.017328691703589985, 0.12149939119053965]


In [19]:
result_above = pyhf.infer.hypotest(
    1.0,
    workspace.data(model_above),
    model_above,
    qtilde=True,
    return_expected_set=True
)
print(f"Observed: {result_above[0]}, Expected band: {[exp.tolist() for exp in result_above[1]]}")

Observed: 0.5856783902261125, Expected band: [0.049880381089905654, 0.12644650624454196, 0.2925782875240953, 0.5694124401526922, 0.8475953637671176]


And as you can see, we're getting results that we generally expect. Excluded models are those for which $\text{CL}_\text{s} < 0.05$. Additionally, you can see that the expected bands $-2\sigma$ for the $(1000, 0)$ point is just slightly below the observed result for the $(650, 0)$ point which is what we observe in the figure above.